<a href="https://colab.research.google.com/github/kartikeyahl/MS_analysis/blob/main/PunchTimeAnalysis7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [2]:
# Importing the excel
# !unzip punch_short_.xlsx
dataset = pd.read_excel('punch_short.xlsx',parse_dates=['Date'],
    date_parser=lambda x: pd.to_datetime(x, format='%Y%m%d'))
dataset2=pd.read_excel('Master Data.xlsx')

In [3]:
#Renaming column name to a valid column name format as per python
dataset.rename(columns={'IN/OUT': 'IN_OUT'}, inplace=True)

In [4]:
#Droping rows that conatain 1 in 'IN_OUT' column
dataset=dataset[dataset.IN_OUT!=1]

In [5]:
#Converting Date column values into year-month-day format
# j,k=0,0
# for i in dataset.iloc[:,3]:
#   dataset.Date[j]=datetime.strptime(str(i),'%Y%m%d').date()
#   j+=1

dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l=[]
for i in range(len(dataset)-1):
  j=i+1
  x=str(dataset.iloc[i]['Time'])
  y=str(dataset.iloc[j]['Time'])

  if len(x)<=5:
    x=x.zfill(6)
  if len(y)<=5:
    y=y.zfill(6)

  x = datetime.strptime(x, "%H%M%S")
  x.strftime("%I:%M %p")
  y = datetime.strptime(y, "%H%M%S")
  y.strftime("%I:%M %p")

  l.append([dataset.iloc[i]['E.Code'], dataset.iloc[i]['Date'].date(), dataset.iloc[i]['IN_OUT'], int(dataset.iloc[i]['Time']), datetime.combine(dataset.iloc[j]['Date'], y.time()) - datetime.combine(dataset.iloc[i]['Date'], x.time())])  
  i=i+2

In [7]:
# #Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)

# x='140000'
# y='020000'
# if len(x)<=4:
#   x=x.zfill(6)
# if len(y)<=4:
#   y=y.zfill(6)

# x2 = datetime.strptime(x, "%H%M%S")
# x2.strftime("%I:%M %p")
# y2 = datetime.strptime(y, "%H%M%S")
# y2.strftime("%I:%M %p")
# print(x2,y2)
# r=datetime.combine(datetime.strptime('20221012','%Y%m%d'), y2.time()) - datetime.combine(datetime.strptime('20221011','%Y%m%d'), x2.time())
# print(r)

In [8]:
#Replacing ":" with "." and getting upto 2 decimal places in Time to make mathematical operations feasible on i
for i in range(len(l)):
  l[i][4] = l[i][4].__str__().replace(":",".")
  s=l[i][4]
  l[i][4]=s[0:-3]

In [9]:
#Removing redundant rows and storing in new data-structure
l2=[]
for i in range(len(l)):
  if 'P20' not in str(l[i][2]):
    l2.append(l[i]) 

In [10]:
for i in range(len(l2)):
  if 'd' in l2[i][4]:
    s=l2[i][4]
    s=float(s[8:len(s)])
    l2[i][4]=24+s

In [11]:
# s='1 day, 0.53'

# s=float(s[5:len(s)])
# s=24+s
# print(s)

In [12]:
#Converting string type to float for Time column
for i in range(len(l2)):
  try:
   l2[i][4]=float("{:.2f}".format(float(l2[i][4])))
  except:
    print(l2[i][0],l2[i][4])

In [13]:
"""Converting time from float to mins, applied logic to check if it is greater than 9hrs & 30mins. Subtacting the 
result to get over time (minutes) converting and storing it into hrs.minutes format. """

l3=[]
for i in range(len(l2)):
  
  frac, whole = math.modf(l2[i][4])
  mins=int(whole*60+frac*100)
  
  if l2[i][2]=='P10' and l2[i][3]>220000:
    # print(frac,whole)
    # print(l2[i][0])
    # print(l2[i][4])
    # print(mins)
    if mins>450:
      mins=mins-450
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  else:
    mins=mins-570
    hours=mins//60
    minutes=mins%60
    if minutes<10:
      ot_time = float("{}.0{}".format(hours, minutes))
    else:
      ot_time = float("{}.{}".format(hours, minutes))
    l2[i].append(ot_time)
    l3.append(l2[i])  

In [14]:
#Removing redundant rows and storing in new data-structure
l4=[]
for i in range(len(l3)):
  if l3[i][5]>0:
    l4.append(l3[i]) 

In [15]:
# Converting 2d list to dataframe to apply CRUD opertations
df = pd.DataFrame(l4, columns=['E.Code','Date','IN/OUT','Time','Total Time','OT'])

In [16]:
#Droping an unnecesary column
df=df.drop(['IN/OUT','Time','Total Time'], axis=1)

In [17]:
df

,E.Code,Date,OT
0,48,2022-09-15,0.27
1,72,2022-09-15,0.06
2,80,2022-09-15,0.09
3,108,2022-09-15,0.39
4,126,2022-09-15,1.11
5,221,2022-09-15,0.22
6,224,2022-09-15,0.03
7,292,2022-09-15,0.17
8,368,2022-09-15,0.11
9,386,2022-09-15,0.03


In [18]:
ll1=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df)):
  # Function call
  result = binarySearch(arr, 0, len(arr)-1, df.iloc[i]['E.Code'])

  if result != -1:
      ll1.append([dataset2.iloc[result]['Full Name'], dataset2.iloc[result]['Designation'],dataset2.iloc[result]['Operation'],dataset2.iloc[result]['Division'],dataset2.iloc[result]['Department']])
  else:
      ll1.append(['NaN','NaN','NaN','NaN'])


In [19]:
# Converting 2d list to dataframe to apply CRUD opertations
dff = pd.DataFrame(ll1, columns=['Full Name','Designation','Operation','Division','Department'])

In [20]:
df_final = pd.concat([df['E.Code'], dff, df[['Date','OT']]], axis=1)

In [21]:
""" Printing dataframe (E.Code, Designation, Opr., Division, Dept., Date, OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes)) 
"""
df_final

,E.Code,Full Name,Designation,Operation,Division,Department,Date,OT
0,48,Sanjul Saxena,General Manager,Customer Service,CS Technical,NaN,2022-09-15,0.27
1,72,Manish Bansal,General Manager,Mass Production Purchase & Supplier Mgmt,General Purchase 1,NaN,2022-09-15,0.06
2,80,M V N Purushottam Sarma,Chief Manager,Narsapura Plant,NaN,Projects,2022-09-15,0.09
3,108,Sanjeev Kumar Singh,General Manager,Quality Control Head Office,Engineering Quality,NaN,2022-09-15,0.39
4,126,Mohan Singh,General Manager,Vithalapur Plant,Production Body Vithalapur,NaN,2022-09-15,1.11
5,221,Kavindra Kumar,Manager,Manesar Plant,Production Control Manesar,Material Service,2022-09-15,0.22
6,224,Ravinder Kumar Sejwal,General Manager,Finance & Accounts,Accounts,NaN,2022-09-15,0.03
7,292,Shiv Prasad Chittora,Manager,Tapukara Plant,Production Power Train Tapukara,Assembly Engine 2F,2022-09-15,0.17
8,368,Atanu Biswas,General Manager,Narsapura Plant,Production Control Narsapura,NaN,2022-09-15,0.11
9,386,Manas Kumar Behera,Manager,Narsapura Plant,Production Body Narsapura,Dy. CIQ - Production Body,2022-09-15,0.03


In [22]:
# df_final.iloc[10035]

In [23]:
""" Applying Groupby opr. to group dataframe on basis of E.Code and apply sum and count opr. to 
compute Total_OT & Total_OT_Days """

groupby_ecode_Total_OT = df.groupby(['E.Code'],as_index=False).sum()
groupby_ecode_Total_OT_Days = df.groupby(['E.Code'],as_index=False).count()
groupby_operation_sum = df_final.groupby(['Operation'],as_index=False).sum()
groupby_operation = df_final.groupby(['Operation'],as_index=False).count()
groupby_division_sum = df_final.groupby(['Division']).sum()
groupby_division = df_final.groupby(['Division']).count()
groupby_designation_sum = df_final.groupby(['Designation']).sum()
groupby_designation = df_final.groupby(['Designation']).count()

In [24]:
#Converting Groupby object into DataFrame to apply CRUD opr.
df2=pd.DataFrame(groupby_ecode_Total_OT)
df3=pd.DataFrame(groupby_ecode_Total_OT_Days)
df4=pd.DataFrame(groupby_operation)
df5=pd.DataFrame(groupby_division)
df6=pd.DataFrame(groupby_operation_sum)
df7=pd.DataFrame(groupby_division_sum)
df8=pd.DataFrame(groupby_designation)
df9=pd.DataFrame(groupby_designation_sum)

In [25]:
df6

,Operation,E.Code,OT
0,Customer Service,48,0.27
1,Finance & Accounts,224,0.03
2,General and Corporate Affairs,1094,2.33
3,Manesar Plant,9099,24.02
4,Mass Production Purchase & Supplier Mgmt,144,0.16
5,Narsapura Plant,7262,6.03
6,New Model Purchase & Parts Strategy,3037,2.01
7,Overseas Business,2447,0.49
8,Quality Control Head Office,1604,1.99
9,Strategic Information System,125,0.36


In [26]:
df4

,Operation,E.Code,Full Name,Designation,Division,Department,Date,OT
0,Customer Service,1,1,1,1,0,1,1
1,Finance & Accounts,1,1,1,1,0,1,1
2,General and Corporate Affairs,2,2,2,2,2,2,2
3,Manesar Plant,12,12,12,12,11,12,12
4,Mass Production Purchase & Supplier Mgmt,2,2,2,2,0,2,2
5,Narsapura Plant,13,13,13,11,7,13,13
6,New Model Purchase & Parts Strategy,5,5,5,5,2,5,5
7,Overseas Business,3,3,3,3,3,3,3
8,Quality Control Head Office,3,3,3,3,0,3,3
9,Strategic Information System,1,1,1,1,0,1,1


In [27]:
df4=df4.drop(['E.Code','Full Name','Designation','Division','Department','Date'], axis=1)
df5=df5.drop(['E.Code','Full Name','Designation','Operation','Department','Date'], axis=1)
df8=df8.drop(['E.Code','Full Name','Operation','Division','Department','Date'], axis=1)
df6=df6.drop(['E.Code'], axis=1)
df7=df7.drop(['E.Code'], axis=1)
df9=df9.drop(['E.Code'], axis=1)

In [28]:
df4.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df5.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df8.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df6.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df7.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [29]:
l4=[]
for i in df2.index:
      frac, whole = math.modf(df2['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      l4.append(ot_time)

In [30]:
llp=[]
for i in df6.index:
      frac, whole = math.modf(df6['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      llp.append(ot_time)

In [31]:
lllp=[]
for i in df7.index:
      frac, whole = math.modf(df7['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      lllp.append(ot_time)

In [32]:
llllp=[]
for i in df9.index:
      frac, whole = math.modf(df9['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      llllp.append(ot_time)

In [33]:
df2['Total OT']=l4
df6['Total OT']=llp
df7['Total OT']=lllp
df9['Total OT']=llllp

In [34]:
#Droping an unnecesary column
df3=df3.drop(['Date'], axis=1)

In [35]:
df2=df2.drop(['OT'], axis=1)
df6=df6.drop(['OT'], axis=1)
df7=df7.drop(['OT'], axis=1)
df9=df9.drop(['OT'], axis=1)

In [36]:
#Renaming Columns
#df2.rename(columns={'OT': 'Total OT'}, inplace=True)
df3.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [37]:
#Creating a final dataframe (E.Code, Total OT_Days, Total OT)
# result = pd.concat([df3,df2], axis=1)
result_2 =  pd.concat([df4,df6], axis=1)
result_3 =  pd.concat([df5,df7], axis=1)
result_4 =  pd.concat([df8,df9], axis=1)

In [38]:
df2

,E.Code,Total OT
0,32,0.22
1,48,0.27
2,49,0.35
3,72,0.16
4,80,0.23
5,108,0.39
6,125,0.36
7,126,1.27
8,168,0.25
9,221,0.55


In [39]:
ll2=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df3)):
  # Function call
  res = binarySearch(arr, 0, len(arr)-1, df3.iloc[i]['E.Code'])

  if res != -1:
      ll2.append([dataset2.iloc[res]['Full Name'], dataset2.iloc[res]['Designation'],dataset2.iloc[res]['Operation'],dataset2.iloc[res]['Division'],dataset2.iloc[res]['Department']])
  else:
      ll2.append(['NaN','NaN','NaN','NaN'])


In [40]:
# Converting 2d list to dataframe to apply CRUD opertations
dfff = pd.DataFrame(ll2, columns=['Full Name','Designation','Operation','Division','Department'])

In [41]:
result = pd.concat([df3['E.Code'], dfff, df3['Total OT_Days'], df2['Total OT']], axis=1)

In [42]:
""" Printing dataframe (E.Code, Total OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes), 
    Total OT_Days) 
"""
result

,E.Code,Full Name,Designation,Operation,Division,Department,Total OT_Days,Total OT
0,32,Sanjeev Juyal,Chief Manager,General and Corporate Affairs,Industrial Relations,Employee Benefit,1,0.22
1,48,Sanjul Saxena,General Manager,Customer Service,CS Technical,NaN,1,0.27
2,49,Atul Goel,General Manager,New Model Purchase & Parts Strategy,Cost & Parts Strategy Frame & Budget,NaN,1,0.35
3,72,Manish Bansal,General Manager,Mass Production Purchase & Supplier Mgmt,General Purchase 1,NaN,2,0.16
4,80,M V N Purushottam Sarma,Chief Manager,Narsapura Plant,NaN,Projects,2,0.23
5,108,Sanjeev Kumar Singh,General Manager,Quality Control Head Office,Engineering Quality,NaN,1,0.39
6,125,Sudhansu Maharana,General Manager,Strategic Information System,Manufacturing Operation Systems 4F,NaN,1,0.36
7,126,Mohan Singh,General Manager,Vithalapur Plant,Production Body Vithalapur,NaN,2,1.27
8,168,Kalyan Shivram Jamdade,Assistant Manager,Vithalapur Plant,Production Body Vithalapur,Paint 4F,1,0.25
9,221,Kavindra Kumar,Manager,Manesar Plant,Production Control Manesar,Material Service,2,0.55


In [43]:
# Sorting dataframe in descending order on basis of Total OT_Days (User can get top x employees on basis of Total OT_Days)
cont2=result.sort_values(by=['Total OT_Days'], ascending=False)

In [44]:
# Sorting dataframe in descending order on basis of Total OT (User can get top x employees on basis of Total OT)
cont3=result.sort_values(by=['Total OT'], ascending=False)

In [45]:
cont4=result_2.sort_values(by=['Total OT_Days'], ascending=False)

In [46]:
cont5=result_2.sort_values(by=['Total OT'], ascending=False)

In [47]:
cont6=result_3.sort_values(by=['Total OT_Days'], ascending=False)

In [48]:
cont7=result_3.sort_values(by=['Total OT'], ascending=False)

In [49]:
cont8=result_4.sort_values(by=['Total OT_Days'], ascending=False)

In [50]:
cont9=result_4.sort_values(by=['Total OT'], ascending=False)

In [51]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
from openpyxl import Workbook
import xlsxwriter
import os

workbook = xlsxwriter.Workbook(r'OT_Analysis.xlsx')
with pd.ExcelWriter(r'OT_Analysis.xlsx') as writer:
#writer = r"C:\Users\Kartikey\Desktop\shorttt.xlsx"
# write dataframe to excel
    df_final.to_excel(writer,sheet_name='OT')
    result.to_excel(writer,sheet_name='Total_OT')
    cont2.to_excel(writer,sheet_name='Descending_Total_OT_Days')
    cont3.to_excel(writer,sheet_name='Descending_Total_OT')
    cont4.to_excel(writer,sheet_name='Operation_Highest_Total_OT_Days')
    cont5.to_excel(writer,sheet_name='Operation_Highest_Total_OT')
    cont6.to_excel(writer,sheet_name='Division_Highest_Total_OT_Days')
    cont7.to_excel(writer,sheet_name='Division_Highest_Total_OT')
    cont8.to_excel(writer,sheet_name='Desig_Highest_Total_OT_Days')
    cont9.to_excel(writer,sheet_name='Desig_Highest_Total_OT')